In [7]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# 음식 풍미 분석용 system prompt
describe_dish_system_prompt = """
 Persona:
            As a flavor analysis system, I am equipped with a deep understanding of food ingredients, cooking methods, and sensory properties such as taste, texture, and aroma. I can assess and break down the flavor profiles of dishes by identifying the dominant tastes (sweet, sour, salty, bitter, umami) as well as subtler elements like spice levels, richness, freshness, and aftertaste. I am able to compare different foods based on their ingredients and cooking techniques, while also considering cultural influences and typical pairings. My goal is to provide a detailed analysis of a dish’s flavor profile to help users better understand what makes it unique or to aid in choosing complementary foods and drinks.

            Role:

            1. Flavor Identification: I analyze the dominant and secondary flavors of a dish, highlighting key taste elements such as sweetness, acidity, bitterness, saltiness, umami, and the presence of spices or herbs.
            2. Texture and Aroma Analysis: Beyond taste, I assess the mouthfeel and aroma of the dish, taking into account how texture (e.g., creamy, crunchy) and scents (e.g., smoky, floral) contribute to the overall experience.
            3. Ingredient Breakdown: I evaluate the role each ingredient plays in the dish’s flavor, including their impact on the dish's balance, richness, or intensity.
            4. Culinary Influence: I consider the cultural or regional influences that shape the dish, understanding how traditional cooking methods or unique ingredients affect the overall taste.
            5. Food and Drink Pairing: Based on the dish's flavor profile, I suggest complementary food or drink pairings that enhance or balance the dish’s qualities.

            Examples:

            - Dish Flavor Breakdown:
            For a butter garlic shrimp, I identify the richness from the butter, the pungent aroma of garlic, and the subtle sweetness of the shrimp. The dish balances richness with a touch of saltiness, and the soft, tender texture of the shrimp is complemented by the slight crispness from grilling.

            - Texture and Aroma Analysis:
            A creamy mushroom risotto has a smooth, velvety texture due to the creamy broth and butter. The earthy aroma from the mushrooms enhances the umami flavor, while a sprinkle of Parmesan adds a savory touch with a mild sharpness.

            - Ingredient Role Assessment:
            In a spicy Thai curry, the coconut milk provides a rich, creamy base, while the lemongrass and lime add freshness and citrus notes. The chilies bring the heat, and the balance between sweet, sour, and spicy elements creates a dynamic flavor profile.

            - Cultural Influence:
            A traditional Italian margherita pizza draws on the classic combination of fresh tomatoes, mozzarella, and basil. The simplicity of the ingredients allows the flavors to shine, with the tanginess of the tomato sauce balancing the richness of the cheese and the freshness of the basil.

            - Food Pairing Example:
            For a rich chocolate cake, I would recommend a sweet dessert wine like Port to complement the bitterness of the chocolate, or a light espresso to contrast the sweetness and enhance the richness of the dessert.
        """
        # 모델의 "뇌 속 역할/성격" 설정

# 음식 풍미 분석 함수
def describe_dish_flavor_chain(query):
    if query.get("image_urls"):
        messages = [
            ("system", describe_dish_system_prompt),
            ("human", [
                {"type": "text", "text": "이 요리의 이름과 맛을 한 문장으로 요약해주세요."},
                *[{"type": "image_url", "image_url": {"url": url}} for url in query["image_urls"]]
            ])
        ]
    else:
        messages = [
            ("system", describe_dish_system_prompt),
            ("user", "요리 이미지가 제공되지 않았습니다. 이미지를 업로드해주세요.")
        ]

    prompt = ChatPromptTemplate.from_messages(messages)
    output_parser = StrOutputParser()
    chain = prompt | llm | output_parser
    return chain.invoke(query)



### 1. 환경 준비 & Pinecone 연결

In [17]:
from dotenv import load_dotenv
import os
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.schema.runnable import RunnableLambda

load_dotenv()

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

#벡터스토어를 이용한 임베딩&적재  
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

vector_store = PineconeVectorStore(
    index_name=("wine-reviews"),
    embedding=embedding,
    pinecone_api_key=os.getenv("PINECONE_API_KEY")
)


### 1-1. 기존에 생성한 인덱스가 있을 시

In [12]:
from pinecone import Pinecone
import os

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index("wine-reviews")   
stats = index.describe_index_stats()
print(stats)

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'wine_reviews_ns1': {'vector_count': 129971}},
 'total_vector_count': 129971,
 'vector_type': 'dense'}


### 2.와인 검색 함수

In [3]:
def search_wine(dish_flavor: str):
    results = vector_store.similarity_search(
        dish_flavor,
        k=5,
        namespace=os.getenv("PINECONE_NAMESPACE")
    )
    return {
        "dish_flavor": dish_flavor,
        "wine_reviews": "\n".join([doc.page_content for doc in results])
    }

### 3. 와인 추천 함수(from_messages 버전)

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def recommend_wine_chain(data):
    chat_template = ChatPromptTemplate.from_messages(
        [
            ("system", "당신은 전문 소믈리에입니다. 음식 풍미와 와인 리뷰를 바탕으로 최적의 와인을 추천하세요."),
            ("human", [
                {"type": "text", "text": f"음식 풍미 설명: {data['dish_flavor']}"},
                {"type": "text", "text": f"참고할 와인 리뷰:\n{data['wine_reviews']}"},
                {"type": "text", "text": "위 정보를 바탕으로 추천 와인과 이유를 알려주세요."}
            ])
        ]
    )
    chain = chat_template | llm | StrOutputParser()
    return chain.invoke({})


### 4.파이프라인 연결

In [18]:
# 22.ipynb에서 describe_dish_flavor_chain을 불러와야 함
# 방법 1: 함수 코드 복사
# 방법 2: %run "./22.와인이미지_랭체인프롬프트.ipynb"

runnable_1 = RunnableLambda(describe_dish_flavor_chain)
runnable_2 = RunnableLambda(search_wine)
runnable_3 = RunnableLambda(recommend_wine_chain)

chain = runnable_1 | runnable_2 | runnable_3

response = chain.invoke({"image_urls": ["https://images.vivino.com/thumbs/Z90I3--JRKWlpMA8wdLY-Q_pb_x600.png"]})
print(response)


음식의 풍미와 제공된 와인 리뷰를 바탕으로, "Masseria del Fauno 2010 Primitivo (Puglia)"를 추천드립니다. 이 와인은 풍부한 과일 향과 부드러운 탄닌이 어우러진 깊고 진한 맛을 제공하며, 특히 크리스마스 향신료, 시나몬, 넛맥, 블랙베리 잼, 플럼 케이크의 달콤한 노트가 두드러집니다. 이러한 풍미는 음식의 풍부한 과일 향과 잘 어울리며, 부드러운 탄닌이 음식의 질감을 보완해 줄 것입니다.

또한, 이 와인은 리뷰에서 87점을 받았으며, 이는 상당히 좋은 평가로, 와인의 품질을 보장합니다. Puglia 지역의 Primitivo는 일반적으로 풍부한 과일 향과 부드러운 탄닌을 특징으로 하며, 이 와인은 그 특성을 잘 살리고 있습니다. 따라서, 이 와인은 음식과 함께 즐기기에 적합한 선택이 될 것입니다.
